# Курсовой проект по курсу Спортивный анализ данных. Платформа Kaggle

In [1]:
from typing import List,Optional
from tqdm import tqdm

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import os

import seaborn as sns
import scipy.stats as st
from scipy.stats import probplot, ks_2samp
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score,train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

import missingno as msno

import xgboost as xgb
import lightgbm as lgb
import catboost as cb

In [2]:
DATA_PATH="Data"
def load_data(dataset_name,data_path=DATA_PATH):
    csv_path = os.path.join(data_path,dataset_name)
    return pd.read_csv(csv_path)

In [3]:
train = load_data("train.csv")
test = load_data("test.csv")
applications_history=load_data("applications_history.csv")
bki=load_data("bki.csv")
client_profile=load_data("client_profile.csv")
payments=load_data("payments.csv")

In [4]:
#Размерности датасетов
print("train.shape = {} rows, {} cols".format(*train.shape))
print("test.shape = {} rows, {} cols".format(*test.shape))

train.shape = 110093 rows, 3 cols
test.shape = 165141 rows, 2 cols


In [5]:
train.describe()

,APPLICATION_NUMBER,TARGET
count,1.100930e+05,110093.000000
mean,1.235772e+08,0.080813
std,8.880939e+04,0.272550
min,1.234233e+08,0.000000
25%,1.235005e+08,0.000000
50%,1.235769e+08,0.000000
75%,1.236539e+08,0.000000
max,1.237308e+08,1.000000


In [6]:
train.head(5)

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE
0,123687442,0,Cash
1,123597908,1,Cash
2,123526683,0,Cash
3,123710391,1,Cash
4,123590329,1,Cash


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110093 entries, 0 to 110092
Data columns (total 3 columns):
APPLICATION_NUMBER    110093 non-null int64
TARGET                110093 non-null int64
NAME_CONTRACT_TYPE    110093 non-null object
dtypes: int64(2), object(1)
memory usage: 2.5+ MB


In [8]:
train["APPLICATION_NUMBER"].nunique()

110093

**Выводы - поле APPLICATION_NUMBER - просто идентификатор типа ID, который нужен лишь для соединения с другими датасетами**

In [9]:
dataset_target = pd.DataFrame(train['TARGET'])

In [10]:
def target_description_print(ds, target_name):                              
    """
    Печать распределения целевых значений.
    
    Parameters
    ----------
    ds: pd.DataFrame
        Вектор целевой переменной.
    target_name: str
        название целевого поля
    """
    
    values_count_TARGET_0 = ds[ds[target_name]==0].count()[target_name]
    values_count_TARGET_1 = ds[ds[target_name]==1].count()[target_name]
    values_count_TARGET_1_2 = np.around(values_count_TARGET_1/values_count_TARGET_0,3)
    values_count_TARGET_Sum = values_count_TARGET_0+values_count_TARGET_1
    print("TARGET==0 values count =",values_count_TARGET_0)
    print("TARGET==1 values count =",values_count_TARGET_1)
    print("(TARGET==1)count / (TARGET==0)count =",values_count_TARGET_1_2)
    print("TARGET==0 and TARGET==1 in sum values count =",values_count_TARGET_Sum)

In [11]:
target_description_print(dataset_target,'TARGET')

TARGET==0 values count = 101196
TARGET==1 values count = 8897
(TARGET==1)count / (TARGET==0)count = 0.088
TARGET==0 and TARGET==1 in sum values count = 110093


**Обработка предыдущих заявок клиента**

In [12]:
applications_history.head()

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,AMOUNT_ANNUITY,AMT_APPLICATION,AMOUNT_CREDIT,AMOUNT_PAYMENT,AMOUNT_GOODS_PAYMENT,NAME_CONTRACT_STATUS,DAYS_DECISION,...,NAME_PRODUCT_TYPE,SELLERPLACE_AREA,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,49298709,123595216,NaN,1730.430,17145.0,17145.0,0.0,17145.0,Approved,73,...,XNA,35,12.0,middle,365243.0,42.0,300.0,42.0,37.0,0.0
1,50070639,123431468,Cash,25188.615,607500.0,679671.0,NaN,607500.0,Approved,164,...,x-sell,-1,36.0,low_action,365243.0,134.0,916.0,365243.0,365243.0,1.0
2,49791680,123445379,Cash,15060.735,112500.0,136444.5,NaN,112500.0,Approved,301,...,x-sell,-1,12.0,high,365243.0,271.0,59.0,365243.0,365243.0,1.0
3,50087457,123499497,Cash,47041.335,450000.0,470790.0,NaN,450000.0,Approved,512,...,x-sell,-1,12.0,middle,365243.0,482.0,152.0,182.0,177.0,1.0
4,49052479,123525393,Cash,31924.395,337500.0,404055.0,NaN,337500.0,Refused,781,...,walk-in,-1,24.0,high,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
print(applications_history.columns.tolist())

['PREV_APPLICATION_NUMBER', 'APPLICATION_NUMBER', 'NAME_CONTRACT_TYPE', 'AMOUNT_ANNUITY', 'AMT_APPLICATION', 'AMOUNT_CREDIT', 'AMOUNT_PAYMENT', 'AMOUNT_GOODS_PAYMENT', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']


In [14]:
apps_groupby = applications_history.groupby("APPLICATION_NUMBER", as_index=False)
previous_app_counts = apps_groupby["PREV_APPLICATION_NUMBER"].count()
previous_app_counts = previous_app_counts.rename(columns={
    "PREV_APPLICATION_NUMBER": "PREV_APPS_COUNT"
})
previous_app_counts.head(n=3)

,APPLICATION_NUMBER,PREV_APPS_COUNT
0,123423340,1
1,123423341,1
2,123423342,3


Оценка полезности новых числовых переменных

In [15]:
train = train.merge(
    previous_app_counts, how="left", on="APPLICATION_NUMBER"
)
train["PREV_APPS_COUNT"] = train["PREV_APPS_COUNT"].fillna(0)
train.head(n=2)

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,PREV_APPS_COUNT
0,123687442,0,Cash,3.0
1,123597908,1,Cash,4.0


In [16]:
def calculate_feature_separating_ability(
    features: pd.DataFrame, target: pd.Series, fill_value: float = -9999) -> pd.DataFrame:
    """
    Оценка разделяющей способности признаков с помощью метрики GINI.

    Parameters
    ----------
    features: pandas.core.frame.DataFrame
        Матрица признаков.

    target: pandas.core.frame.Series
        Вектор целевой переменной.

    fill_value: float, optional, default = -9999
        Значение для заполнения пропусков в значении признаков.
        Опциональный параметр, по умолчанию, равен -9999;

    Returns
    -------
    scores: pandas.core.frame.DataFrame
        Матрица важности признаков.

    """
    scores = {}
    for feature in features:
        score = roc_auc_score(
            target, features[feature].fillna(fill_value)
        )
        scores[feature] = 2*score - 1

    scores = pd.Series(scores)
    scores = scores.sort_values(ascending=False)

    return scores

In [17]:
numerical_applications_history = applications_history.select_dtypes(exclude=["object"])
numerical_applications_history=numerical_applications_history[numerical_applications_history["APPLICATION_NUMBER"].isin(train["APPLICATION_NUMBER"].values)]
numerical_applications_history=numerical_applications_history.groupby("APPLICATION_NUMBER", as_index=False).median()

In [18]:
numerical_applications_history

,APPLICATION_NUMBER,PREV_APPLICATION_NUMBER,AMOUNT_ANNUITY,AMT_APPLICATION,AMOUNT_CREDIT,AMOUNT_PAYMENT,AMOUNT_GOODS_PAYMENT,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,123423341,48307032.0,9251.775,179055.000,179055.00,0.000,179055.00,606.0,500.0,24.0,365243.0,565.0,125.0,25.0,17.0,0.0
1,123423345,49346257.0,26636.760,270000.000,267930.00,34840.170,394708.50,181.0,-1.0,15.0,365243.0,413.0,737.0,182834.0,182829.5,0.0
2,123423349,49617703.0,27463.410,247212.000,260811.00,0.000,247212.00,1070.0,8636.0,10.0,365243.0,1039.0,769.0,769.0,762.0,0.0
3,123423351,49472223.5,9482.355,54360.000,68985.00,0.000,90000.00,669.5,-1.0,18.0,365243.0,802.0,112.0,147.0,144.0,1.0
4,123423352,48676397.0,6538.185,36742.500,36083.25,3375.000,51030.00,564.5,14.5,10.0,365243.0,826.0,224.0,346.0,344.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104651,123730828,49219711.0,10760.220,112500.000,133528.50,0.000,116102.25,1335.0,-1.0,14.0,365243.0,1424.0,756.5,1488.0,1483.5,0.5
104652,123730830,49091897.5,53643.330,1170000.000,1271929.50,4.095,1170000.00,1500.5,0.0,36.0,365243.0,1135.0,644.0,985.0,977.5,0.5
104653,123730833,49113857.0,6516.045,58482.000,64656.00,0.000,58482.00,565.0,168.0,12.0,365243.0,534.0,204.0,1970.0,1967.0,0.0
104654,123730838,48867053.0,19865.835,59395.500,63576.00,4155.750,495000.00,343.0,-1.0,36.0,365243.0,735.5,1018.5,1018.5,1009.0,0.0


In [19]:
numerical_applications_history = numerical_applications_history.merge(train["APPLICATION_NUMBER"], how="right", on="APPLICATION_NUMBER")

scores = calculate_feature_separating_ability(
    numerical_applications_history, train["TARGET"]
)
scores.head(n=10)

AMOUNT_PAYMENT               0.011738
DAYS_DECISION                0.006577
PREV_APPLICATION_NUMBER      0.006401
NFLAG_INSURED_ON_APPROVAL    0.006115
AMT_APPLICATION              0.004255
AMOUNT_CREDIT                0.004160
DAYS_FIRST_DUE               0.003278
DAYS_FIRST_DRAWING           0.002790
SELLERPLACE_AREA             0.001685
AMOUNT_ANNUITY               0.001034
dtype: float64

Вывод: AMOUNT_PAYMENT это поле информативно и значимо и будем его использовать              

In [20]:
cols_drop=numerical_applications_history.columns.tolist()
cols_drop=[x for x in cols_drop if x not in ['AMOUNT_PAYMENT', 'APPLICATION_NUMBER']]
print(cols_drop)

['PREV_APPLICATION_NUMBER', 'AMOUNT_ANNUITY', 'AMT_APPLICATION', 'AMOUNT_CREDIT', 'AMOUNT_GOODS_PAYMENT', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']


In [21]:
numerical_applications_history=numerical_applications_history.drop(cols_drop, axis=1)

In [22]:
train = train.merge(numerical_applications_history, how="left", on="APPLICATION_NUMBER")
train["AMOUNT_PAYMENT"] = train["AMOUNT_PAYMENT"].fillna(0)
train.head(n=2)

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,PREV_APPS_COUNT,AMOUNT_PAYMENT
0,123687442,0,Cash,3.0,7168.5
1,123597908,1,Cash,4.0,0.0


**Обработка данных БКИ о предыдущих кредитах клиента**

In [23]:
bki

,APPLICATION_NUMBER,BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,123538884,5223613,Active,currency 1,718.0,0,377.0,NaN,19386.810,0,675000.00,320265.495,0.0,0.0,Consumer credit,39.0,NaN
1,123436670,6207544,Closed,currency 1,696.0,0,511.0,511.0,0.000,0,93111.66,0.000,0.0,0.0,Consumer credit,505.0,NaN
2,123589020,6326395,Closed,currency 1,165.0,0,149.0,160.0,NaN,0,36000.00,0.000,0.0,0.0,Consumer credit,150.0,0.0
3,123494590,6606618,Active,currency 1,55.0,0,310.0,NaN,NaN,0,38664.00,37858.500,NaN,0.0,Consumer credit,15.0,NaN
4,123446603,5046832,Active,currency 1,358.0,0,35.0,NaN,NaN,0,67500.00,0.000,0.0,0.0,Credit card,116.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945229,123673441,5235365,Closed,currency 1,2759.0,0,1298.0,1834.0,NaN,0,332725.50,0.000,NaN,0.0,Consumer credit,1707.0,NaN
945230,123539211,5899696,Active,currency 1,359.0,0,1467.0,NaN,NaN,0,1471500.00,1320183.000,0.0,0.0,Consumer credit,47.0,NaN
945231,123686333,5445504,Closed,currency 1,1102.0,0,725.0,370.0,NaN,0,112500.00,0.000,0.0,0.0,Consumer credit,233.0,NaN
945232,123508200,6679628,Active,currency 1,1579.0,0,2085.0,NaN,2339.955,0,108000.00,0.000,0.0,0.0,Credit card,16.0,NaN


In [24]:
numerical_bki = bki.select_dtypes(exclude=["object"])
numerical_bki=numerical_bki[numerical_bki["APPLICATION_NUMBER"].isin(train["APPLICATION_NUMBER"].values)]
numerical_bki=numerical_bki.groupby("APPLICATION_NUMBER", as_index=False).median()

In [25]:
numerical_bki

,APPLICATION_NUMBER,BUREAU_ID,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,123423341,6171940.0,1042.0,0.0,432.5,967.0,20.25,0.0,120735.00,0.00,0.000,0.0,34.0,0.0
1,123423352,5965158.5,2037.5,0.0,1308.0,1276.0,19305.00,0.0,391770.00,NaN,NaN,0.0,1274.5,0.0
2,123423355,5245900.0,141.0,0.0,223.0,NaN,NaN,0.0,88996.50,63724.50,0.000,0.0,8.0,NaN
3,123423359,6109409.0,232.0,0.0,56.0,135.0,0.00,0.0,22680.00,2922.75,0.000,0.0,92.5,NaN
4,123423370,6266842.0,203.0,0.0,960.0,153.0,NaN,0.0,548779.50,NaN,NaN,0.0,72.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84503,123730827,5485899.0,1296.0,0.0,584.0,1052.0,0.00,0.0,605161.62,0.00,0.000,0.0,1028.0,NaN
84504,123730828,6074180.0,571.0,0.0,467.0,513.0,0.00,0.0,27981.00,0.00,0.000,0.0,468.0,0.0
84505,123730830,6785214.5,1296.5,0.0,958.0,1300.0,NaN,0.0,776880.00,0.00,247496.445,0.0,167.0,100818.0
84506,123730833,5709132.0,1011.0,0.0,816.0,NaN,NaN,0.0,675000.00,416983.50,0.000,0.0,103.0,NaN


Оценка полезности новых числовых переменных

In [26]:
numerical_bki = numerical_bki.merge(train["APPLICATION_NUMBER"], how="right", on="APPLICATION_NUMBER")

scores = calculate_feature_separating_ability(
    numerical_bki, train["TARGET"]
)
scores.head(n=10)

DAYS_CREDIT               0.018200
BUREAU_ID                 0.015481
AMT_CREDIT_SUM            0.014376
AMT_CREDIT_SUM_DEBT       0.013831
DAYS_CREDIT_UPDATE        0.013042
AMT_CREDIT_SUM_OVERDUE    0.012580
CREDIT_DAY_OVERDUE        0.012542
CNT_CREDIT_PROLONG        0.012044
DAYS_CREDIT_ENDDATE       0.011707
DAYS_ENDDATE_FACT         0.011377
dtype: float64

Вывод:  поля: 

DAYS_CREDIT              
BUREAU_ID                 
AMT_CREDIT_SUM            


информативны и значимы и будем их использовать  

In [28]:
cols_drop=numerical_bki.columns.tolist()
cols_drop=[x for x in cols_drop if x not in ['APPLICATION_NUMBER','DAYS_CREDIT', 'BUREAU_ID','AMT_CREDIT_SUM']]
print(cols_drop)

['CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY']


In [29]:
numerical_bki=numerical_bki.drop(cols_drop, axis=1)
numerical_bki=numerical_bki.fillna(0)
numerical_bki

,APPLICATION_NUMBER,BUREAU_ID,DAYS_CREDIT,AMT_CREDIT_SUM
0,123423341,6171940.0,1042.0,120735.0
1,123423352,5965158.5,2037.5,391770.0
2,123423355,5245900.0,141.0,88996.5
3,123423359,6109409.0,232.0,22680.0
4,123423370,6266842.0,203.0,548779.5
...,...,...,...,...
110088,123689525,0.0,0.0,0.0
110089,123464177,0.0,0.0,0.0
110090,123450128,0.0,0.0,0.0
110091,123595479,0.0,0.0,0.0


In [30]:
train = train.merge(numerical_bki, how="left", on="APPLICATION_NUMBER")

train.head(n=2)

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,PREV_APPS_COUNT,AMOUNT_PAYMENT,BUREAU_ID,DAYS_CREDIT,AMT_CREDIT_SUM
0,123687442,0,Cash,3.0,7168.5,5640206.0,2562.0,56362.5
1,123597908,1,Cash,4.0,0.0,5964293.0,560.0,137910.6


**Обработка данных клиентский профиль, некоторые знания, которые есть у компании о клиенте**

In [31]:
client_profile

,APPLICATION_NUMBER,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123666076,F,0,157500.0,270000.0,13500.0,Incomplete higher,Civil marriage,0.008068,8560,...,2.0,0.329471,0.236315,0.678568,0.0,0.0,0.0,0.0,1.0,2.0
1,123423688,F,0,270000.0,536917.5,28467.0,Secondary / secondary special,Married,0.020246,23187,...,2.0,NaN,0.442295,0.802745,0.0,0.0,0.0,0.0,1.0,1.0
2,123501780,M,1,427500.0,239850.0,23850.0,Incomplete higher,Married,0.072508,14387,...,3.0,0.409017,0.738159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,123588799,M,0,112500.0,254700.0,17149.5,Secondary / secondary special,Married,0.019101,14273,...,2.0,NaN,0.308994,0.590233,0.0,0.0,0.0,0.0,0.0,3.0
4,123647485,M,0,130500.0,614574.0,19822.5,Lower secondary,Married,0.022625,22954,...,2.0,NaN,0.739408,0.156640,0.0,0.0,1.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,123657254,M,0,216000.0,45000.0,2425.5,Higher education,Married,0.018850,19150,...,2.0,0.555436,0.581592,0.048259,0.0,0.0,0.0,0.0,1.0,3.0
249996,123645397,M,0,103500.0,675000.0,28507.5,Higher education,Married,0.014520,19604,...,2.0,NaN,0.676409,0.726711,0.0,0.0,0.0,0.0,0.0,0.0
249997,123504053,M,0,202500.0,1078200.0,38331.0,Secondary / secondary special,Single / not married,0.031329,8351,...,1.0,NaN,0.353665,0.283712,0.0,0.0,0.0,0.0,1.0,4.0
249998,123547316,F,0,135000.0,500211.0,38839.5,Secondary / secondary special,Married,0.030755,13277,...,2.0,0.305746,0.682462,0.639708,0.0,0.0,0.0,0.0,0.0,3.0


In [32]:
numerical_client_profile = client_profile.select_dtypes(exclude=["object"])
numerical_client_profile=numerical_client_profile[numerical_client_profile["APPLICATION_NUMBER"].isin(train["APPLICATION_NUMBER"].values)]
numerical_client_profile=numerical_client_profile.groupby("APPLICATION_NUMBER", as_index=False).median()

In [33]:
numerical_client_profile

,APPLICATION_NUMBER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123423341,0,202500.000,406597.5,24700.5,0.018801,9461,637,NaN,1,...,1.0,0.083037,0.262949,0.139376,0.0,0.0,0.0,0.0,0.0,1.0
1,123423345,0,121500.000,513000.0,21865.5,0.028663,19932,3038,NaN,0,...,1.0,NaN,0.322738,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,123423351,1,112500.000,652500.0,21177.0,0.022800,10197,679,NaN,0,...,3.0,0.319760,0.651862,0.363945,0.0,0.0,0.0,1.0,0.0,0.0
3,123423352,0,38419.155,148365.0,10678.5,0.015221,20417,365243,NaN,1,...,2.0,0.722044,0.555183,0.652897,0.0,0.0,0.0,0.0,0.0,2.0
4,123423355,0,189000.000,773680.5,32778.0,0.010006,14583,203,NaN,0,...,2.0,0.721940,0.642656,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89534,123730828,0,117000.000,1071909.0,31473.0,0.010147,23125,365243,NaN,0,...,2.0,NaN,0.307082,0.255332,0.0,0.0,0.0,0.0,1.0,8.0
89535,123730830,0,225000.000,521280.0,23089.5,0.014464,16471,286,NaN,0,...,2.0,NaN,0.615261,0.397946,0.0,0.0,0.0,0.0,1.0,2.0
89536,123730833,0,585000.000,1575000.0,43443.0,0.028663,20965,1618,2.0,0,...,2.0,NaN,0.599173,0.365617,0.0,0.0,0.0,0.0,0.0,0.0
89537,123730838,0,180000.000,270126.0,12028.5,0.007020,16679,1222,0.0,0,...,2.0,0.251096,0.651306,0.524496,0.0,0.0,0.0,0.0,0.0,6.0


Оценка полезности новых числовых переменных

In [34]:
numerical_client_profile = numerical_client_profile.merge(train["APPLICATION_NUMBER"], how="right", on="APPLICATION_NUMBER")

scores = calculate_feature_separating_ability(
    numerical_client_profile, train["TARGET"]
)
scores.head(n=10)

EXTERNAL_SCORING_RATING_2     0.016773
AMT_REQ_CREDIT_BUREAU_YEAR    0.016579
REGION_POPULATION             0.016136
CHILDRENS                     0.015849
FAMILY_SIZE                   0.015761
DAYS_ON_LAST_JOB              0.015241
TOTAL_SALARY                  0.013819
AGE                           0.012451
AMOUNT_CREDIT                 0.011560
AMT_REQ_CREDIT_BUREAU_DAY     0.010084
dtype: float64

Вывод:  поля: 

EXTERNAL_SCORING_RATING_2  
AMT_REQ_CREDIT_BUREAU_YEAR  
REGION_POPULATION             


информативны и значимы и будем их использовать  

In [35]:
cols_drop=numerical_client_profile.columns.tolist()
cols_drop=[x for x in cols_drop if x not in ['APPLICATION_NUMBER','EXTERNAL_SCORING_RATING_2', 'AMT_REQ_CREDIT_BUREAU_YEAR','REGION_POPULATION']]
print(cols_drop)
numerical_client_profile=numerical_client_profile.drop(cols_drop, axis=1)
numerical_client_profile=numerical_client_profile.fillna(0)
numerical_client_profile

['CHILDRENS', 'TOTAL_SALARY', 'AMOUNT_CREDIT', 'AMOUNT_ANNUITY', 'AGE', 'DAYS_ON_LAST_JOB', 'OWN_CAR_AGE', 'FLAG_PHONE', 'FLAG_EMAIL', 'FAMILY_SIZE', 'EXTERNAL_SCORING_RATING_1', 'EXTERNAL_SCORING_RATING_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT']


,APPLICATION_NUMBER,REGION_POPULATION,EXTERNAL_SCORING_RATING_2,AMT_REQ_CREDIT_BUREAU_YEAR
0,123423341,0.018801,0.262949,1.0
1,123423345,0.028663,0.322738,0.0
2,123423351,0.022800,0.651862,0.0
3,123423352,0.015221,0.555183,2.0
4,123423355,0.010006,0.642656,0.0
...,...,...,...,...
110088,123447470,0.000000,0.000000,0.0
110089,123477663,0.000000,0.000000,0.0
110090,123560225,0.000000,0.000000,0.0
110091,123458312,0.000000,0.000000,0.0


In [36]:
train = train.merge(numerical_client_profile, how="left", on="APPLICATION_NUMBER")

train

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,PREV_APPS_COUNT,AMOUNT_PAYMENT,BUREAU_ID,DAYS_CREDIT,AMT_CREDIT_SUM,REGION_POPULATION,EXTERNAL_SCORING_RATING_2,AMT_REQ_CREDIT_BUREAU_YEAR
0,123687442,0,Cash,3.0,7168.500,5640206.0,2562.0,56362.5,0.019101,0.645914,2.0
1,123597908,1,Cash,4.0,0.000,5964293.0,560.0,137910.6,0.000000,0.000000,0.0
2,123526683,0,Cash,6.0,18814.500,6203921.0,1952.0,360000.0,0.026392,0.682149,4.0
3,123710391,1,Cash,2.0,2250.000,6635818.0,2509.0,119731.5,0.031329,0.171299,0.0
4,123590329,1,Cash,9.0,0.000,5726448.0,1028.0,102910.5,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
110088,123458312,0,Cash,5.0,0.000,6693249.0,1625.0,819000.0,0.000000,0.000000,0.0
110089,123672463,0,Cash,13.0,3316.500,5143430.5,696.5,92250.0,0.010032,0.471774,2.0
110090,123723001,0,Cash,2.0,4281.750,6138700.0,1945.0,43740.0,0.000000,0.000000,0.0
110091,123554358,0,Cash,2.0,0.000,6764230.0,683.0,96192.0,0.019101,0.651227,1.0


**Обработка история платежей клиента**

In [37]:
payments

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,49011181,123664960,1.0,5,1002.0,1015.0,12156.615,12156.615
1,48683432,123497205,1.0,13,442.0,432.0,18392.535,10047.645
2,48652024,123749925,1.0,10,8.0,23.0,5499.945,5499.945
3,48398897,123550846,0.0,82,398.0,398.0,7082.145,7082.145
4,49867197,123562174,0.0,63,1359.0,1359.0,156.735,156.735
...,...,...,...,...,...,...,...,...
1023927,50029793,123728077,0.0,123,993.0,993.0,2700.000,2700.000
1023928,48418780,123568892,0.0,73,529.0,529.0,232.335,232.335
1023929,49942303,123494001,2.0,24,389.0,393.0,23284.485,23284.485
1023930,50081462,123609565,0.0,4,2671.0,2671.0,9000.000,9000.000


Здесь имеет смысл взять не медиану а сумму по группе APPLICATION_NUMBER

In [38]:
numerical_payments = payments.select_dtypes(exclude=["object"])
numerical_payments=numerical_payments[numerical_payments["APPLICATION_NUMBER"].isin(train["APPLICATION_NUMBER"].values)]
numerical_payments=numerical_payments.groupby("APPLICATION_NUMBER", as_index=False).sum()

In [39]:
numerical_payments

,APPLICATION_NUMBER,PREV_APPLICATION_NUMBER,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,123423341,96614064,2.0,25,440.0,474.0,18503.550,18503.550
1,123423345,99026173,2.0,10,586.0,634.0,31510.440,31510.440
2,123423351,198361844,10.0,29,1808.0,1942.0,32600.790,32600.790
3,123423352,638512843,4.0,651,13724.0,13775.0,97347.240,97092.495
4,123423355,48519261,1.0,7,837.0,844.0,14480.460,14476.500
...,...,...,...,...,...,...,...,...
81962,123730828,973693012,16.0,351,26824.0,26975.0,273476.475,314893.980
81963,123730830,148410562,6.0,15,3449.0,3487.0,222889.275,222889.275
81964,123730833,246370751,5.0,26,7338.0,7410.0,43654.680,43654.680
81965,123730838,147559050,2.0,16,1363.0,1365.0,35209.575,35209.575


In [40]:
numerical_payments = numerical_payments.merge(train["APPLICATION_NUMBER"], how="right", on="APPLICATION_NUMBER")

scores = calculate_feature_separating_ability(
    numerical_payments, train["TARGET"]
)
scores.head(n=10)

NUM_INSTALMENT_VERSION     0.014857
AMT_PAYMENT                0.012068
AMT_INSTALMENT             0.011075
DAYS_ENTRY_PAYMENT         0.008761
DAYS_INSTALMENT            0.008617
NUM_INSTALMENT_NUMBER      0.008067
PREV_APPLICATION_NUMBER    0.007592
APPLICATION_NUMBER        -0.007666
dtype: float64

Вывод:  поля: 
        
NUM_INSTALMENT_VERSION     
AMT_PAYMENT              
AMT_INSTALMENT        

информативны и значимы и будем их использовать  

In [42]:
cols_drop=numerical_payments.columns.tolist()
cols_drop=[x for x in cols_drop if x not in ['APPLICATION_NUMBER','NUM_INSTALMENT_VERSION', 'AMT_PAYMENT','AMT_INSTALMENT']]
print(cols_drop)
numerical_payments=numerical_payments.drop(cols_drop, axis=1)
numerical_payments=numerical_payments.fillna(0)
numerical_payments

[]


,APPLICATION_NUMBER,NUM_INSTALMENT_VERSION,AMT_INSTALMENT,AMT_PAYMENT
0,123423341,2.0,18503.55,18503.550
1,123423345,2.0,31510.44,31510.440
2,123423351,10.0,32600.79,32600.790
3,123423352,4.0,97347.24,97092.495
4,123423355,1.0,14480.46,14476.500
...,...,...,...,...
110088,123450259,0.0,0.00,0.000
110089,123525515,0.0,0.00,0.000
110090,123457895,0.0,0.00,0.000
110091,123708679,0.0,0.00,0.000


In [43]:
train = train.merge(numerical_payments, how="left", on="APPLICATION_NUMBER")

train

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,PREV_APPS_COUNT,AMOUNT_PAYMENT,BUREAU_ID,DAYS_CREDIT,AMT_CREDIT_SUM,REGION_POPULATION,EXTERNAL_SCORING_RATING_2,AMT_REQ_CREDIT_BUREAU_YEAR,NUM_INSTALMENT_VERSION,AMT_INSTALMENT,AMT_PAYMENT
0,123687442,0,Cash,3.0,7168.500,5640206.0,2562.0,56362.5,0.019101,0.645914,2.0,4.0,25103.700,15841.485
1,123597908,1,Cash,4.0,0.000,5964293.0,560.0,137910.6,0.000000,0.000000,0.0,1.0,11349.900,11349.900
2,123526683,0,Cash,6.0,18814.500,6203921.0,1952.0,360000.0,0.026392,0.682149,4.0,2.0,80677.890,80677.890
3,123710391,1,Cash,2.0,2250.000,6635818.0,2509.0,119731.5,0.031329,0.171299,0.0,1.0,2450.115,2450.115
4,123590329,1,Cash,9.0,0.000,5726448.0,1028.0,102910.5,0.000000,0.000000,0.0,4.0,42433.560,42433.560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110088,123458312,0,Cash,5.0,0.000,6693249.0,1625.0,819000.0,0.000000,0.000000,0.0,3.0,14176.890,14176.890
110089,123672463,0,Cash,13.0,3316.500,5143430.5,696.5,92250.0,0.010032,0.471774,2.0,5.0,36582.075,38241.990
110090,123723001,0,Cash,2.0,4281.750,6138700.0,1945.0,43740.0,0.000000,0.000000,0.0,1.0,4998.645,4998.645
110091,123554358,0,Cash,2.0,0.000,6764230.0,683.0,96192.0,0.019101,0.651227,1.0,0.0,0.000,0.000


In [44]:
target_name='TARGET'
target=train[target_name]
data=train.drop(["TARGET"], axis=1)

In [45]:
x_train, x_valid = train_test_split(
    data, train_size=0.8, random_state=1
)
y_train, y_valid = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))

x_train.shape = 88074 rows, 13 cols
x_valid.shape = 22019 rows, 13 cols


In [46]:
def get_categorical_features(ds):
    categorical_features = ds.select_dtypes(include=["object"])
    categorical_features=categorical_features.fillna('')
    cat_feats =[]
    for col in categorical_features.columns:
        cat_feats.append(col)
        
    return cat_feats

In [47]:
cat_feats=get_categorical_features(train)

In [48]:
data

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,PREV_APPS_COUNT,AMOUNT_PAYMENT,BUREAU_ID,DAYS_CREDIT,AMT_CREDIT_SUM,REGION_POPULATION,EXTERNAL_SCORING_RATING_2,AMT_REQ_CREDIT_BUREAU_YEAR,NUM_INSTALMENT_VERSION,AMT_INSTALMENT,AMT_PAYMENT
0,123687442,Cash,3.0,7168.500,5640206.0,2562.0,56362.5,0.019101,0.645914,2.0,4.0,25103.700,15841.485
1,123597908,Cash,4.0,0.000,5964293.0,560.0,137910.6,0.000000,0.000000,0.0,1.0,11349.900,11349.900
2,123526683,Cash,6.0,18814.500,6203921.0,1952.0,360000.0,0.026392,0.682149,4.0,2.0,80677.890,80677.890
3,123710391,Cash,2.0,2250.000,6635818.0,2509.0,119731.5,0.031329,0.171299,0.0,1.0,2450.115,2450.115
4,123590329,Cash,9.0,0.000,5726448.0,1028.0,102910.5,0.000000,0.000000,0.0,4.0,42433.560,42433.560
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110088,123458312,Cash,5.0,0.000,6693249.0,1625.0,819000.0,0.000000,0.000000,0.0,3.0,14176.890,14176.890
110089,123672463,Cash,13.0,3316.500,5143430.5,696.5,92250.0,0.010032,0.471774,2.0,5.0,36582.075,38241.990
110090,123723001,Cash,2.0,4281.750,6138700.0,1945.0,43740.0,0.000000,0.000000,0.0,1.0,4998.645,4998.645
110091,123554358,Cash,2.0,0.000,6764230.0,683.0,96192.0,0.019101,0.651227,1.0,0.0,0.000,0.000


In [49]:
cb_params = {
    "n_estimators": 4000,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 30,
    "verbose": 300,
    "max_depth": 7,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 1350,
    "thread_count": 8,
    "random_seed": 42,
    "use_best_model": True,
    "learning_rate": 0.05    
}

In [50]:
model_cb = cb.CatBoostClassifier(**cb_params)
model_cb.fit(x_train, 
             y_train, 
             eval_set=[(x_train, y_train), (x_valid, y_valid)], 
             cat_features=cat_feats,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.6077639	test1: 0.6007525	best: 0.6007525 (0)	total: 168ms	remaining: 11m 10s
300:	test: 0.6885272	test1: 0.6327828	best: 0.6335883 (254)	total: 3.68s	remaining: 45.2s
600:	test: 0.7177451	test1: 0.6318550	best: 0.6335883 (254)	total: 7.2s	remaining: 40.7s
900:	test: 0.7401044	test1: 0.6295714	best: 0.6335883 (254)	total: 10.5s	remaining: 36.3s
1200:	test: 0.7610695	test1: 0.6276005	best: 0.6335883 (254)	total: 13.9s	remaining: 32.5s
1500:	test: 0.7809965	test1: 0.6251617	best: 0.6335883 (254)	total: 17.5s	remaining: 29.1s
Stopped by overfitting detector  (1350 iterations wait)

bestTest = 0.6335882684
bestIteration = 254

Shrink model to first 255 iterations.


In [51]:
from sklearn.preprocessing import OneHotEncoder

def handle_categorical_data(data, categorical_features):
    enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
    for col in categorical_features.columns:
        ohe_ftrs = enc.fit_transform(categorical_features[col].values.reshape(-1,1))
        tmp = pd.DataFrame(ohe_ftrs, columns = [col + '_' + str(i) for i in range(ohe_ftrs.shape[1])])
        data = pd.concat([data, tmp], axis=1)
    return data

In [52]:
def get_data_with_numerical_features(ds, missings_filling_mark="mean"):
    numerical_features = ds.select_dtypes(exclude=["object"])
    print(f"count of numeric_features {numerical_features.shape[1]}")
    numerical_features = numerical_features.drop([ "TARGET"], axis=1)
    #Заменяем пропуски на среднии
    if missings_filling_mark == "mean":
        numerical_features = numerical_features.fillna(numerical_features.mean())
    else:
        numerical_features = numerical_features.fillna(-1)
    
    return numerical_features
    

In [53]:
def get_data_with_OHE_categorical_features(ds,target_name,handle_numerical_features:callable,encoder:callable):
    data=handle_numerical_features(ds)
    categorical_features = ds.select_dtypes(include=["object"])
    data = encoder(data, categorical_features)
    target=ds[target_name]
    return data,target

In [54]:
data,target=get_data_with_OHE_categorical_features(train,'TARGET',handle_numerical_features=get_data_with_numerical_features,
                                            encoder=handle_categorical_data)

count of numeric_features 13


In [55]:
x_train, x_valid = train_test_split(
    data, train_size=0.8, random_state=1
)
y_train, y_valid = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))

x_train.shape = 88074 rows, 14 cols
x_valid.shape = 22019 rows, 14 cols


In [56]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 10000,
    "n_jobs": 8,
    "seed": 27,
    "early_stopping_rounds":300
}

In [57]:
model_lgb = lgb.LGBMClassifier(**params)
model_lgb.fit(
    X=x_train,
    y=y_train,
    eval_set=[(x_train, y_train), (x_valid, y_valid)],
    early_stopping_rounds=300,
    eval_metric="auc",
    verbose=500,
    categorical_feature="auto"
)

C:\Users\Windows\.conda\envs\tf\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Windows\.conda\envs\tf\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] early_stopping_round is set=300, early_stopping_rounds=300 will be ignored. Current value: early_stopping_round=300
[500]	training's auc: 0.723568	valid_1's auc: 0.632264


LGBMClassifier(early_stopping_rounds=300, learning_rate=0.01, metric='auc',
               n_estimators=10000, n_jobs=8, objective='binary', seed=27)